<a href="https://colab.research.google.com/github/VCosmo/VCosmo_Repo/blob/main/run_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
from scipy.integrate import cumtrapz
import scipy

from gal4H0 import *

In [ ]:
cosmo=FlatLambdaCDM(H0=70.,Om0=0.25)

In [ ]:
import pandas as pd
# Define the path to the catalog compressed CSV file:
catalog_filename = "/home/Virgo_Lab/13826.csv.bz2"
# Define the list of columns that uniquely identify each row
data = pd.read_csv(catalog_filename, sep=",", comment='#', na_values=r'\N', compression='bz2')
abs_M=data['abs_mag_r'].to_numpy()[0:1000:]
galaxies_list=data['z'].to_numpy()[0:1000:]
Labs=M2L(abs_M)
app_m=M2m(abs_M, cosmo.luminosity_distance(galaxies_list).to('Mpc').value)
print(len(app_m))

In [ ]:
import os, sys
sys.path.append('../')

np.random.seed(0)

filename=['6_0_10perc_acc.p' ,'6_1_10perc_acc.p' ,'6_2_10perc_acc.p']

beta=1
betas=[beta-1, beta, beta+1]

Ngw=100
sigma_dl=0.1
zcut_rate=1.4
dl_thr=1550
H0_array=np.linspace(40,120,2000)
Nrep=20

for j in range(3):

  output={'H0_grid':H0_array,
       'single_pos':[],
       'true_H0':np.zeros(Nrep)}

  for ii in tqdm(range(Nrep)):
    output['true_H0'][ii]=np.random.uniform(40,120,size=1)
    true_cosmology = FlatLambdaCDM(H0=output['true_H0'][ii],Om0=0.25)
    gw_obs_dl,_,_,std_dl=draw_gw_events_L(Ngw,sigma_dl,dl_thr,galaxies_list,true_cosmology,zcut_rate, Labs, beta)
    posterior_matrix,combined=galaxy_catalog_analysis_accurate_redshift_L(H0_array,galaxies_list,zcut_rate,gw_obs_dl,sigma_dl,dl_thr,app_m,betas[j])
    output['single_pos'].append(posterior_matrix)

  pickle.dump(output,open(filename[j],'wb'))